# **Autoregressive conditional heteroskedasticity model (ARCH)**

## Importing Relevant Libraries

In [ ]:

!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras torch torchvision \
    tqdm scikit-image pmdarima arch --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import *
sns.set()

## Loading the Data

In [ ]:
# !gdown https://drive.google.com/uc?id=1y6TP7bJqdrnWDi1gHqT2__yN1xLox6_K

In [ ]:
raw_csv_data = pd.read_csv('Index2018.csv')
raw_csv_data

In [ ]:
df_comp = raw_csv_data.copy()

## Preprocessing the Data

In [ ]:
df_comp.date = pd.to_datetime(df_comp.date, dayfirst= True)
df_comp.set_index('date', inplace = True)
df_comp = df_comp.drop(['spx', 'dax', 'nikkei'], axis = 1)
df_comp

In [ ]:
df_comp['market_value'] = df_comp.ftse
del df_comp['ftse']

In [ ]:
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method = 'ffill')
df_comp

## Splitting the Data

In [ ]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [ ]:
df.tail()

In [ ]:
df_test.head()

## LLR Test Function

In [ ]:
from scipy.stats.distributions import chi2

In [ ]:
def LLR_test(mod_1, mod_2, DF=1):
    L1 = mod_1.fit().llf
    L2 = mod_2.fit().llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR,DF).round(3)
    return p

## Augmented Dickey-Fuller Test

In [ ]:
import statsmodels.tsa.stattools as sts

In [ ]:
sts.adfuller(df.market_value)

From the DF test we see that the market value prices for FTSE100 belong to a Non-Stationery Source, hence it is a better option to Extract the "Returns", check for stationarity and then work on them

## Computing and Storing Returns

Percentage change between the current and a prior element.

Computes the percentage change from the immediately previous row by default. This is useful in comparing the percentage of change in a time series of elements.

In [ ]:
df['returns'] = df.market_value.pct_change(1).mul(100)

## Computing and Storing Squared Returns

In [ ]:
df['sq_returns'] = df.returns.mul(df.returns)

## Returns v/s Squared returns

In [ ]:
df.returns.plot(figsize = (20,5))
plt.title("Returns", size = 24)
plt.show()

In [ ]:
df.sq_returns.plot(figsize = (20,5))
plt.title("Volatility", size = 22)
plt.show()

## PACF

In [ ]:
import statsmodels.graphics.tsaplots as sgt

In [ ]:
sgt.plot_pacf(df.returns[1:], lags = 40, zero = False, method = ('ols'))
plt.title('PACF Returns', size = 22)
plt.show()

From the PACF for Volatility we can see that the first 6 cofficients are significant with the first 5 yeilding between 0.15 and 0.25

Such High significant values of partal autocorrelation among the first few lags suggests that there tend to be short term trends in variance
Another way of looking at it would be clustering. This suggests that there are periods of High Variation followed by periods of High Variation; Similarly,  there are periods of Low Variation followed by periods ofd Low Variation

In [ ]:
sgt.plot_pacf(df.sq_returns[1:], lags = 40, zero = False, method = ('ols'))
plt.title("PACF Volatility", size = 22)
plt.show()

## The arch_model() Method

In [ ]:
# !pip install arch -U

In [ ]:
from arch import arch_model

This value just controls how frequently you get output. 5 indicates only report every 5th iteration. 1 would report each iteration. 0 turns reporting off.

In [ ]:
model_arch_1 = arch_model(df.returns[1:])
results_arch_1 = model_arch_1.fit(update_freq = 5)
results_arch_1.summary()

## The Simple ARCH(1) Model

In [ ]:
model_arch_1 = arch_model(df.returns[1:], 
                          mean = "Constant",
                          vol = "ARCH",
                          p = 1)
results_arch_1 = model_arch_1.fit(update_freq = 5)
results_arch_1.summary()


## Higher Lag ARCH Models

## ARCH(2)

In [ ]:
model_arch_2 = arch_model(df.returns[1:], mean = "Constant", vol = "ARCH", p = 2)
results_arch_2 = model_arch_2.fit(update_freq = 5)
results_arch_2.summary()

## ARCH(3)

In [ ]:
model_arch_3 = arch_model(df.returns[1:], mean = "Constant", vol = "ARCH", p = 3)
results_arch_3 = model_arch_3.fit(update_freq = 5)
results_arch_3.summary()

# **Generalized autoregressive conditional heteroskedasticity model(GARCH)**

In [ ]:
from arch import arch_model

## Simple GARCH(1,1) Model

In [ ]:
model_garch_1_1 = arch_model(df.returns[1:], mean = "Constant", vol = "GARCH", p = 1, q = 1)
results_garch_1_1 = model_garch_1_1.fit(update_freq = 5)
results_garch_1_1.summary()

Iteration:      5,   Func. Count:     39,   Neg. LLF: 6972.734736835859
Iteration:     10,   Func. Count:     73,   Neg. LLF: 6970.088042271316
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6970.058366189893
            Iterations: 13
            Function evaluations: 91
            Gradient evaluations: 13


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13948.1
Method:            Maximum Likelihood   BIC:                           13974.2
                                        No. Observations:                 5020
Date:                Sat, May 22 2021   Df Residuals:                     5019
Time:                        07:28:01   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.183e-02      3.939  8.187e-05 [2.342e-02,6.981e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0109  3.004e-03      3.640  2.724e-04 [5.048e-03,1.682e-02]
alpha[1]       0.0835  1.071e-02      7.794  6.476e-15   [6.249e-02,  0.104]
beta[1]        0.9089  1.148e-02     79.168      0.000     [  0.886,  0.931]
============================================================================

Covariance estimator: robust
"""

We do not need to fit higher order GARCH models as the GARCH(1,1) model has been mathematically proven to be the best fit for market returns. Higher order GARCH models in this case will result in Non-Significant Variables

#**Related Articles:**

> * [Date Time Parsing with Pandas](https://analyticsindiamag.com/datetime-parsing-with-pandas/)